# Documentation for:
### 1.Data crawller: Image, dish name, ingredient and nutrition
### 2.First round formatting: separation of value and ingredient name from ingredient data
### 3.Ingredient keyword list construction
### 4.Conversion: ingredient value consistency
### 5.Conversion: ingredient name consistency
### 6.Dimension Reduction: reducing vector length based on frequency count
### 7.Dataset finalisation
### 8.Nutrition calculator

## 0.1.Load Packages

In [ ]:
import requests
import html
import os
import json
import ast
import io
import pandas as pd
import numpy as np
from PIL import Image
from lxml import etree
from PIL import UnidentifiedImageError
from bs4 import BeautifulSoup as BS
from google.colab import drive
drive.mount('/content/drive')
from collections import Counter
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# 1.Data crawller

## 1.1.Get dish url


In [ ]:
# Get the main category URL of dishes on the allrecipes website.
def category_url_extraction(ALL_url):
  url = ALL_url
  kv = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
  page = requests.get(url, headers = kv)
  soup = BS(page.text, 'html')
  urls = soup.find_all('a',class_="link-list__link type--dog-bold type--dog-link")
  url_list = []

  # Get all links on this page.
  for url in urls:
    url_ = (url['href'])
    url_list.append(url_)
  return url_list

Category_url_list = category_url_extraction('https://www.allrecipes.com/recipes-a-z-6735880')
print(len(Category_url_list))

378


In [ ]:
# Get all dish links under the main category.
def dish_url_extraction(ALL_url):
  url = ALL_url
  page = requests.get(url)
  soup = BS(page.text, 'html')
  main_urls = soup.find_all('a' , class_="comp card--image-top mntl-card-list-items mntl-document-card mntl-card card card--no-image")
  Sub_urls = soup.find_all('a' , class_="comp mntl-card-list-items mntl-document-card mntl-card card card--no-image")
  dish_url_list = []
  for url in main_urls:
    url_ = (url['href'])
    dish_url_list.append(url_)
  for url in Sub_urls:
    url_ = (url['href'])
    dish_url_list.append(url_)
  return dish_url_list

In [ ]:
# Clean up all the URLs of dishes to ensure that there are no duplicates.
def all_dish_url_list():
  all_list = []
  for i in Category_url_list:
    category_all_url_list = dish_url_extraction(i)
    all_list.append(category_all_url_list )
  flattened_list = [item for sublist in all_list for item in sublist]
  no_duplicate_list = []
  for i in flattened_list:
    if i not in no_duplicate_list:
      no_duplicate_list.append(i)
  return no_duplicate_list

all_url_list = all_dish_url_list()
print(len(all_url_list))

14442


## 1.2.Collect img and transform to metrix

In [ ]:
# Open a Public git where only store DATA
!git clone https://github.com/Jeffwuzh/Capstone-C35-2.git
!ls Capstone-C35-2

import pandas as pd
path = "Capstone-C35-2/Dish_list.xlsx"
data = pd.read_excel(path)

Cloning into 'Capstone-C35-2'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 6), reused 38 (delta 2), pack-reused 0
Receiving objects: 100% (47/47), 7.86 MiB | 14.88 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Dish_list.csv	FoodData_Central_foundation_food_csv_2023-04-20  README.md
Dish_list.xlsx	Models


In [ ]:
# Save the matrix-form image to a JSON file.
def save_pic(url, name):
  global Ndish
  file_path = '/content/drive/My Drive/dishimg/%s.json'%(name)
  dish = requests.get(url, timeout=100).content
  try:
    image_data = Image.open(io.BytesIO(dish))
    # Normalization
    new_size = (245, 245)
    resized_image = image_data.resize(new_size)
    resized_img_array = np.array(resized_image)
    img_list = resized_img_array.tolist()
    json_str = json.dumps(img_list)
    with open(file_path, 'w') as f:
      f.write(json_str)
      print('successful: %s'%(name))
  except UnidentifiedImageError as e: # Prevent images from being unable to open.
    Ndish.append(name)
    print(f"Can not find the picture：{e}")

In [ ]:
Mdish = [] # Store as the dish name obtained from the picture.

# Get the images from the dish links.
def picture_extraction(URL):
  url = URL
  global Mdish
  kv = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
  page_text = requests.get(url, headers = kv).text
  tree = etree.HTML(page_text)
  soup = BS(page_text, 'html')
  q = soup.find_all('h1', class_="comp type--lion article-heading mntl-text-block")
  try:
    name = [span.text.replace('\n', ' ').strip() for span in q][0]
    if name in data['dish name'].values:
      try:
        picture = tree.xpath('//*[@id="figure-article_1-0"]/div/div/img/@src')[0]
        save_pic(picture, name)
      except IndexError: # Prevent index errors in HTML.
        try:
          picture = tree.xpath('//*[@id="gallery-photos_1-0"]/div/img/@data-src')[0]
          save_pic(picture, name)
        except IndexError: # Prevent index errors in HTML.
          Mdish.append(name)
          print('Not find picture:%s'%(name))
  except IndexError:
    print("Can not find name")

successful: Air Fryer Smoked Salmon Wontons
None


In [ ]:
# Batch image acquisition
for i in all_url_list[13001:13597]:
  picture_extraction(i)

print(Mdish)

## 1.3.Collect dish ingredients and nutritions

In [ ]:
# Get dish name
def name_extraction(URL):
  url = URL
  page = requests.get(url)
  soup = BS(page.text, 'html')
  q = soup.find_all('h1', class_="comp type--lion article-heading mntl-text-block")
  span_list = [span.text.replace('\n', ' ').strip() for span in q] # Remove whitespace
  return span_list

In [ ]:
# Get dish ingredients
def ingredient_extraction(URL):
  url = URL
  page = requests.get(url)
  soup = BS(page.text, 'html')
  q = soup.find_all('ul', class_="mntl-structured-ingredients__list")
  span_list = [span.text.replace('\n', ' ').strip() for span in q] # Remove whitespace
  return span_list

In [ ]:
# Get dish nutritions
def nutrition_extraction(URL):
  url = URL
  page = requests.get(url)
  soup = BS(page.text, 'html')
  nutrition = soup.find_all('td')
  span_list = [a.text.replace('\n', ' ').strip() for a in nutrition]
  nutrition = []
  quantity = []
  name = []
  for i in span_list:
    if '%' not in str(i):
      nutrition.append(i)
  nutrition = nutrition[8:]
  for i in nutrition:
    unit = i.split()
    quantity.append(unit[-1])
    name.append(i.replace(unit[-1], ''))

  return name, quantity

In [ ]:
# Collect all information into a list.
def information_extraction():
  Name_list = []
  ingredient_list = []
  nutrition_name = []
  nutrition_quantity = []
  for i in Finalised_URL_list[0:1000]:
    name = name_extraction(i)[0]
    Name_list.append(name)
    ingredient = ingredient_extraction(i)
    ingredient_list.append(ingredient)
    name, quantity = nutrition_extraction(i)
    nutrition_name.append(name)
    nutrition_quantity.append(quantity)
  return Name_list, ingredient_list, nutrition_name,  nutrition_quantity

Names,ingredient,nutrition_name,nutrition_quantity = information_extraction()

In [ ]:
#list to dataframe conversion for export purposes
df = pd.DataFrame({'dish name':Names, 'recipe':ingredient,'nutrition_name':nutrition_name, 'nutrition_quantity':nutrition_quantity})
print(df)

excel_file_path = '/content/drive/My Drive/data/14001-14421.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")

# 2.Data formatting(separation of value and ingredient name)

## 2.1 Loading dataset from Approach 1

In [1]:
# Open a Public git where only store DATA
!git clone https://github.com/Jeffwuzh/Capstone-C35-2.git

Cloning into 'Capstone-C35-2'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 51 (delta 7), reused 43 (delta 3), pack-reused 0
Receiving objects: 100% (51/51), 7.89 MiB | 17.75 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!ls Capstone-C35-2

 Dish_list.csv	  FoodData_Central_foundation_food_csv_2023-04-20  'mapping data.xlsx'	 README.md
 Dish_list.xlsx  'ingredient frequency.txt'			    Models


In [3]:
import pandas as pd
path = "Capstone-C35-2/Dish_list.xlsx"
data = pd.read_excel(path)

Checking the data

In [4]:
data

,dish name,recipe,nutrition_name,nutrition_quantity
0,Air Fryer Hearts of Palm Sticks,['1/4 cup all-purpose flour 1/4 teaspoon sal...,"['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['7g', '2g', '98mg', '1497mg', '37g', '6g', '1..."
1,Air Fryer Cheesy Bacon Ranch French Fries,['4 cups frozen French fries 1/4 cup olive o...,"['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['29g', '5g', '7mg', '1253mg', '61g', '5g', '2..."
2,Air Fryer Mini Dark Chocolate Cake with Brown ...,"['cooking spray 1/4 cup flour, plus more for...","['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['16g', '8g', '57mg', '286mg', '46g', '1g', '3..."
3,Air Fryer Spanakopita,"['2 (10-ounces) pkg. frozen chopped spinach, ...","['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['7g', '4g', '16mg', '194mg', '12g', '1g', '0g..."
4,Air Fryer Pecan Crusted Trout,['2/3 cup finely chopped pecans 1 teaspoon d...,"['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['24g', '3g', '130mg', '359mg', '9g', '2g', '3..."
...,...,...,...,...
13592,Mexican Pizza I,['1 (16 ounce) can refried beans 1 pound gro...,"['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['48g', '23g', '150mg', '1542mg', '42g', '10g'..."
13593,Pizza Cupcakes,['cooking spray 8 medium cherry tomatoes 1...,"['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['4g', '1g', '8mg', '435mg', '21g', '1g', '3g'..."
13594,Easy Mini Bagel Pizzas,"['8 mini bagels, split ¼ cup pizza sauce ...","['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['6g', '3g', '33mg', '788mg', '30g', '1g', '1g..."
13595,Authentic Pizza Margherita,['3 ½ cups all-purpose flour 1 teaspoon salt...,"['Total Fat ', 'Saturated Fat ', 'Cholesterol ...","['37g', '16g', '90mg', '2187mg', '106g', '6g',..."


## 2.2 Extract the Ingredients and unit

Setting the progress into function

In [5]:
units = [
    'ounces', 'cups', 'teaspoons', 'tablespoons', 'sheets', 'pounds','leaves', 'sprays', 'buns', 'cloves',
    'eggs', 'tablets', 'spoons', 'pinches', 'dashes', 'sticks','slices', 'cans', 'packages', 'bottles',
    'jars', 'boxes', 'bags', 'quarts', 'pints', 'gallons', 'lbs','slice', 'liters', 'dishes',
    'ounce', 'cup', 'teaspoon', 'tablespoon', 'sheet', 'pound', 'spray', 'bun', 'clove',
    'egg', 'tablet', 'spoon', 'pinch', 'dash', 'stick', 'can', 'package', 'bottle',
    'jar', 'box', 'bag', 'quart', 'pint', 'gallon', 'lb', 'liter', 'dish',
    'oz', 'ml', 'litre', 'l ', 'tsp', 'tsp.', 'Tbsp', 'Tbsp.', 'pkg.', 'pkg'
]

In [6]:
import pandas as pd
import re

def extract_amounts_and_ingredients(recipe_str, units):
    recipe_str = str(recipe_str)[2:-2] if str(recipe_str).startswith("[") else str(recipe_str)
    ingredients_list = recipe_str.split('   ')

    amount_unit_re = (
        r'(\d+\s*\(?\d+-?\d*\s*' + r'[^)]+' + r'\)?'  # Matching patterns like (10-ounces)
        r'|\d+\s*\d+\/\d+'                           # Matching patterns like 4 1/2
        r'|\d+\.\d+'                                 # Matching decimal numbers like 0.2
        r'|\d+\/\d+'                                # Matching patterns like 1/2
        r'|\d+\s*½|½'                               # Matching patterns like 4 ½ and just ½
        r'|\d+|⅓|⅔|¼|¾|⅕|⅖|⅗|⅘|⅙|⅚|⅛|⅐|⅑|⅒)'      # Matching other fractions and single numbers
        r'\s*(' + '|'.join([re.escape(unit) for unit in units]) + r'|\blarge\b|\bmedium\b|\bsmall\b)?'
    )

    amounts_units = []
    ingredients = []

    for ingredient in ingredients_list:
        match = re.search(amount_unit_re, ingredient)
        if match:
            group1 = match.group(1) if match.group(1) else ''
            group2 = match.group(2) if match.group(2) else ''
            amounts_units.append(f"{group1} {group2}".strip())

            ingredient_remainder = ingredient[match.end():].strip()
            ingredients.append(ingredient_remainder)
        else:
            amounts_units.append('NA')
            ingredients.append(ingredient)

    return amounts_units, ingredients

# Using apply:
result = data['recipe'].apply(lambda x: extract_amounts_and_ingredients(x, units))
data['amount_units'] = result.apply(lambda x: x[0])
data['ingredients'] = result.apply(lambda x: x[1])

In [7]:
data['ingredients']

0        [all-purpose flour, salt, freshly ground black...
1        [frozen French fries, olive oil, ranch salad d...
2        [cooking spray, flour, plus more for dusting c...
3        [frozen chopped spinach, thawed and squeezed d...
4        [finely chopped pecans, dried rosemary, crushe...
                               ...                        
13592    [refried beans, ground beef, taco seasoning mi...
13593    [cooking spray, cherry tomatoes, refrigerated ...
13594    [mini bagels, split, pizza sauce, shredded piz...
13595    [all-purpose flour, salt, water, active dry ye...
13596    [bacon, cut into 1-inch pieces, chopped onion,...
Name: ingredients, Length: 13597, dtype: object

In [8]:
data['recipe'][2]

"['cooking spray   1/4 cup flour, plus more for dusting cake pan   3 tablespoons unsweetened Dutch process cocoa powder   1/2 teaspoon baking powder   1/4 teaspoon baking soda   1/4 teaspoon salt   3 tablespoons low-fat buttermilk   3 tablespoons unsweetened applesauce   4 1/2 teaspoons canola oil   1 teaspoon instant espresso coffee powder   1/2 teaspoon vanilla extract   1 large egg   1/2 cup sugar   1 ounce dark chocolate, chopped, plus more for garnish  5 tablespoons butter   2 tablespoons honey   1/2 teaspoon vanilla extract   3/4 cup powdered sugar   2 teaspoons water, or as needed']"

In [9]:
data['amount_units'][13586]

['1 (0.25 ounce)',
 '1 cup',
 '3 cups',
 '2 tablespoons',
 '¼ teaspoon',
 '½ cup',
 '1 (6 ounce) can',
 '¾ cup',
 '1 teaspoon',
 '½ teaspoon',
 '½ teaspoon',
 '¼ teaspoon',
 '¼ teaspoon',
 '¼ teaspoon',
 '¼ teaspoon',
 '¼ teaspoon',
 '⅛ teaspoon',
 '⅛ teaspoon']

In [10]:
data['ingredients'][13586]

['envelope active dry yeast',
 'lukewarm water',
 'all-purpose flour',
 'shortening',
 'salt  1 tablespoon vegetable oil',
 'chopped onion',
 'tomato paste',
 'water',
 'salt',
 'white sugar',
 'dried oregano',
 'garlic powder',
 'dried basil',
 'dried marjoram',
 'ground cumin',
 'chili powder',
 'crushed red pepper flakes',
 'ground black pepper  1 (12 ounce) package shredded mozzarella cheese, or to taste']

# 3.Ingredient Keyword list Construction

## 3.1.key words list

In [ ]:
print(len(Words))

620


In [ ]:
#Confirmed ingredient categories
Words =['flour', 'salt', 'black pepper', 'egg', 'bread crumbs', 'parmesan cheese', 'hearts of palm', 'olive oil',
        'parmesan-style cheese', 'quince', 'fresh cranberries', 'mixed berriesfrench fries', 'ranch', 'bacon',
        'cheddar cheese', 'cocoa powder', 'baking powder', 'baking soda', 'milk', 'container blackberries',
        'strawberriesapplesauce', 'canola oil', 'coffee powder', 'nil', 'sugar', 'chocolate', 'honey', 'water',
        'spinach', 'feta cheese', 'mozzarella cheese', 'garlic', 'dough', 'butter', 'tzatziki sauce', 'dill',
        'pecan', 'rosemary', 'trout', 'maple syrup', 'lemon', 'cayenne pepper', 'paprika', 'oregano', 'thyme',
        'cumin', 'onion', 'mayonnaise', 'horseradish sauce', 'ketchup', 'egg whites', 'season', 'vegetable',
        'dipping sauce', 'cracker crumbs', 'cream cheese', 'sour cream', 'lemon juice', 'caramel ice cream',
        'potato', 'ginger', 'garam masala', 'frozen peas', 'cilantro', 'wonton wrappers', 'yogurt', 'lime zest',
        'lime juice', 'zucchini', 'summer squash', 'cremini mushrooms', 'cauliflower florets', 'bell peppers', 'parsley',
        'red peppers', 'red wine vinegar', 'apricot', 'soy sauce', 'orange zest', 'orange juice', 'crabmeat', 'chive',
        'worcestershire sauce', 'cooking spray', 'mustard', 'broccoli', 'salmon', 'raspberry', 'cashew', 'pastry',
        'green beans', 'chili powder', 'celery', 'relish', 'yellow mustard', 'sriracha sauce', 'whole-grain bread',
        'lettuce', 'chicken tenders', 'chicken breasts', 'white pepper', 'basil', 'chicken wings', 'cornstarch',
        'whipped cream', 'white bread', 'ham', 'swiss cheese', 'brussels sprouts', 'waffle', 'shredded cheese', 'banana',
        'cinnamon', 'black-eyed peas', 'tilapia fillets', 'chickpea', 'avocado oil', 'chicken breast', 'chocolate-hazelnut',
        'strawberry jam', 'sour cherries', 'halloumi cheese', 'chicken thighs', 'white vinegar', 'chicken strips',
        'seasoning blend', 'chicken drummies', 'barbecue sauce', 'peach', 'vanilla ice cream', 'pizza sauce', 'pepperoni',
        'caper', 'round brie cheese', 'mojo criollo marinade', 'skewer', 'hot dog buns', 'hot dogs', 'ground beef',
        'ground pork', 'asparagus', 'peppercorn', 'red pepper', 'mushroom', 'pork chops', 'pepper', 'aluminum foil',
        'pasta', 'shrimp', 'steak', 'steakhouse marinade', 'oil', 'cornmeal', 'jalapeno peppers', 'mozzarella',
        'jalapeno pepper', 'chicken', 'kimchi', 'provolone cheese', 'havarti cheese', 'italian sausage', 'tomato',
        'orzo', 'chicken broth', 'tomato paste', 'sliced jalapeno', 'corn chips', 'cabbage', 'rice wine vinegar',
        "red pepper', 'peanut oil", 'yellow pepper', 'snow peas', 'ice cubes', 'pineapple juice', 'passionfruit juice',
        'grape juice', 'green tea extract', 'pineapple', 'whipping cream', 'blueberry', 'blackberry', 'cucumber', 'lime',
        'cherry', 'olive', 'limeade', 'macaroni', 'margarine', 'jalapeno', 'chicken meatballs', 'barbeque sauce', 'carrot',
        'herbes de provence', 'nutmeg', 'hoagie rolls', 'salami', 'crumbled feta', 'emon', 'turkey', 'breadcrumb',
        'ground chicken', 'sake', 'panko breadcrumbs', 'oyster sauce', 'rice', 'mango', 'rum', 'agave syrup',
        'cotija cheese', 'corn kernels', 'chicken soup', 'apple', 'strawberry', 'green pepper', 'baguette', 'sausage',
        'jalapeno cheese', 'ricotta cheese', 'prosciutto', 'beer', 'almond extract', 'ground nutmeg', 'oat', 'almond',
        'blackraspberries', 'heavy cream', 'edamame', 'radish', 'rice vinegar', 'cake mix', 'mint', 'shallot', 'pesto',
        'emon, juiced', 'pepitas', 'pinto beans', 'poppy seeds', 'bun', 'seasonbutter', 'maple extract', 'bourbon',
        'prosecco, well chilled', 'cointreau liqueur', 'sesame seeds', 'sparkling wine', 'marmalade', 'coriander',
        'corn tortillas', 'marinara sauce', 'white wine', 'cream', 'tortilla', 'hot sauce', 'balsamic vinegar', 'curry powder',
        'collard green', 'turmeric', 'salad dressing', 'bay', 'meat', 'mixed herbs', 'cream of tartar', 'cake',
        'xanthan gum', 'yolk', 'cranberry', 'plant', 'italian cheese', 'taloupe', 'gruyère cheese', 'cauliflower', 'tuna',
        'dinner rolls', 'white wine vinegar', 'italian-style salad dressing', 'artichoke hearts', 'burrata cheese', 'raisin',
        'pine nuts', 'tortellini', 'anchovy fillets', 'canning jars', 'breadstick', 'italian herbs', 'balsamic glaze',
        'red wine', 'sage', 'ground sirloin', 'tortilla chips', 'pasteurized cheese', 'frozen corn', 'bbq sauce', 'guacamole',
        'cheetos', 'hawaiian rolls', 'jack cheese', 'biscuit', 'ears corn', 'mesquite wood chips', 'crab legs', 'red grapes',
        'black beans', 'wrapper', 'sauerkraut', 'brie cheese', 'beef chuck roast', 'beef stock', 'merlot wine',
        'chipotle chile powderchuck roast', 'tamari', 'liquid stevia', 'beef bouillon', 'arrowroot powder',
        'tip round roast', 'beef eye of round roast', 'dry sherry', 'beef broth', 'bottom round roast',
        'beef sirloin roast', 'brown gravy mix', 'carbonated beverage', 'sirloin roast', 'beef consomme',
        'au jus mix', 'beef pot roast', 'browning sauce', 'vermouth', 'crescent rolls', 'tofu', 'obster tail',
        'imessherry', 'tarragon', 'lard', 'lobster tails', 'pie crust', 'cornbread mix', 'pot roast', 'fish stock', 'lime','lemon'
        'halibut', 'scallop', 'green peas', 'pork tenderloin', 'coleslaw', 'pork trimmings', 'unflavored gelatin',
        'brown gravy', 'marjoram', 'kaiser rolls', 'bok choy', 'pork shoulder', 'orange, zested', 'fennel seeds',
        'anchovy fillet', 'parchment paper', 'tomatillo', 'pork butt', 'barbeque rub', 'smoke flavoring',
        'pork spareribs', 'chili paste', 'pork loin chops', 'marsala wine', 'teriyaki sauce', 'italian dressing',
        'stuffing mix', 'bread', 'soup base', 'pork ribs', 'back ribs', 'chipotle chiles', 'adobo sauce',
        'pork tenderloins', 'fig', 'date', 'popped popcorn', 'wheat cereal', 'peanut', 'pretzel sticks', 'corn syrup',
        'marshmallow', 'popcorn, popped', 'furikake', 'wasabi peas', 'sumac powder', 'unpopped popcorn', 'chow mein noodles',
        'candy sprinkles', 'lentil', 'soft drink mix', 'buffalo sauce', 'popcorn', 'polenta', 'mascarpone cheese', 'hazelnut',
        'hazelnut liqueur', 'gruyere cheese', 'ancho chile powder', 'romano cheese', 'italian-style cheese', 'yeast',
        'gelatin powder', 'pizza toppings', 'cheese blend', 'anchovy', 'bean', 'salsa', 'beef', 'caraway seeds', 'pumpkin puree',
        'walnut', 'wax beans', 'lima beans', 'baked beans', 'chili beans', 'fennel seed', 'corona beans', 'great northern beans',
        'molasses', 'red beans', 'bean cooking liquid', 'baked beans with pork', 'kidney beans', 'strawberrieskidney beans',
        'garbanzo beans', 'navy beans', 'wheat gluten', 'wheat germ', 'flaxseed', 'instant coffee', 'lining paper', 'cheese food',
        'alfredo sauce', 'vegan cheese', 'green chiles', 'red chile', 'pear', 'pomegranate', 'orange', 'rhubarb', 'star anise',
        'clove', 'cheesecloth', 'mixed fruit', 'tapioca', 'pie shells', 'blackberriespie shell', 'food coloring', 'nori',
        'sriracha chile sauce', 'avocado', 'french herb blend', 'sweet rolls', 'corned beef', 'coconut', 'bread rolls', 'chili sauce',
        'scallion', 'maraschino cherries', 'herb blend', 'crust pies', 'pumpkin pie spice', 'neufchatel cheese', 'cottage cheese',
        'canned pumpkin', 'bagel thins', 'soft drink', 'pizza crust', 'fontina cheese', 'radicchio', 'buffalo wing sauce', 'paper bag',
        'cereal snack mix', 'pretzel', 'corn puffssplit peas', 'chinese five-spice powder', 'pork rib', 'mesquite chips',
        'pork spare ribs', 'lager', 'caraway seed', 'pork loin', 'pork neck bones', 'guajillo chiles', 'boston butt roast', 'whiskey',
        'creme fraiche', 'pork picnic roast', 'baking mix', 'chile paste', 'hoisin sauce', 'sandwich roll', 'pork roast',
        'smooth pate', 'kernel corn', 'corn muffin mix', 'ground allspice', 'turnip', 'black coffee', 'whole allspice berries',
        'yellow squash', 'pico de gallo', 'au jus gravy mix', 'lightly beaten', 'sweet pickles', 'eek', 'matzo meal', 'blackberries',
        'sweet pickle juice', 'processed cheese', 'yellow corn', 'spaghetti sauce', 'artichoke', 'prime rib', 'rib roast',
        'barbeque spice', 'mexico chiles', 'pulled pork', 'enchilada sauce', 'chipotle chile powder', 'frozen blueberries',
        'pumpkin seeds', 'sweetener', 'vegan pie shell', 'blackeyed peas', 'fresh cherries', 'italianstyle cheese', 'gouda',
        'guanciale', 'parmigiano cheese', 'corn puffs', 'pheasant', 'american cheese', 'vinegar', 'shorten', 'wood chips',
        'pumpkin guts', 'sunflower seeds', 'whipped topping', 'tart shells', 'cardamom', 'tangerine', 'gummy candies',
        'club soda', 'sec liqueur', 'tea powder', 'black tea', 'green chilies', 'dulce de leche', 'pitted prunes',
        'fresh strawberries', 'corn truffles', 'oaxaca cheese', 'nacho cheese sauce', 'sweet corn', 'mexican blend cheese',
        'poblano chile', 'swiss chard', 'kale', 'beet', 'quinoa', 'liquid aminos', 'galangal', 'salam', 'tahini', 'arugula',
        'pita', 'lamb shoulder', 'ras el hanout', 'pistachio', 'ravioli', 'seaweed', 'serrano chiles', 'chia seeds',
        'macadamia nuts', 'sunchoke', 'papaya', 'ning jars', 'cider vinegar', 'mussel', 'acorn', 'asiago cheese',
        'cardamom pods', 'noodle', 'ale', 'tater tots', 'seitan', 'island dressing', 'pisco', 'corn husks', 'roast pork',
        'spaghetti', 'maple flavored extract', 'huckleberry', 'chipotle powder', 'skinon mahi mahi', 'cooker liner', 'uette',
        'vacuum bag', 'ettuce', 'mexican crema', 'jackfruit', 'sriracha', 'clam', 'jicama', 'calamari rings', 'traditional sauce',
        'brewed coffee', 'chardonnay wine', 'morel', 'silver nonpareils', 'stevia blend', 'wasabi paste', 'kombu', 'cheese curds',
        'chihuaua cheese', 'masa harina', 'guajillo chile sauce', 'mexicanstyle corn', "blackberries", 'yuca root']

In [ ]:
print(len(key_words))
print(key_words)

619
['flour', 'salt', 'black pepper', 'egg', 'bread crumbs', 'parmesan cheese', 'hearts of palm', 'olive oil', 'parmesan-style cheese', 'quince', 'fresh cranberries', 'mixed berriesfrench fries', 'ranch', 'bacon', 'cheddar cheese', 'cocoa powder', 'baking powder', 'baking soda', 'milk', 'container blackberries', 'strawberriesapplesauce', 'canola oil', 'coffee powder', 'nil', 'sugar', 'chocolate', 'honey', 'water', 'spinach', 'feta cheese', 'mozzarella cheese', 'garlic', 'dough', 'butter', 'tzatziki sauce', 'dill', 'pecan', 'rosemary', 'trout', 'maple syrup', 'lemon', 'cayenne pepper', 'paprika', 'oregano', 'thyme', 'cumin', 'onion', 'mayonnaise', 'horseradish sauce', 'ketchup', 'egg whites', 'season', 'vegetable', 'dipping sauce', 'cracker crumbs', 'cream cheese', 'sour cream', 'lemon juice', 'caramel ice cream', 'potato', 'ginger', 'garam masala', 'frozen peas', 'cilantro', 'wonton wrappers', 'yogurt', 'lime zest', 'lime juice', 'zucchini', 'summer squash', 'cremini mushrooms', 'cauli

## 3.2.Frequency count

In [ ]:
#for key word pre-processing:
def Text_pre_process(Word_list):
  lemmatizer = WordNetLemmatizer()
  def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
  standardized_Words = []
  for i in Word_list:
      i = i.lower().strip()
      pos_tags = nltk.pos_tag([i])
      lemmatized_item = lemmatizer.lemmatize(i, get_wordnet_pos(pos_tags[0][1]))
      standardized_Words.append(lemmatized_item)
  print(standardized_Words)
  word_counts = Counter(standardized_Words)
  key_words = list(word_counts.keys())
  key_words = [item for item in key_words if item != '']
  return key_words

key_words = Text_pre_process(Words)


['flour', 'salt', 'black pepper', 'egg', 'bread crumbs', 'parmesan cheese', 'hearts of palm', 'olive oil', 'parmesan-style cheese', 'quince', 'fresh cranberries', 'mixed berriesfrench fries', 'ranch', 'bacon', 'cheddar cheese', 'cocoa powder', 'baking powder', 'baking soda', 'milk', 'container blackberries', 'strawberriesapplesauce', 'canola oil', 'coffee powder', 'nil', 'sugar', 'chocolate', 'honey', 'water', 'spinach', 'feta cheese', 'mozzarella cheese', 'garlic', 'dough', 'butter', 'tzatziki sauce', 'dill', 'pecan', 'rosemary', 'trout', 'maple syrup', 'lemon', 'cayenne pepper', 'paprika', 'oregano', 'thyme', 'cumin', 'onion', 'mayonnaise', 'horseradish sauce', 'ketchup', 'egg whites', 'season', 'vegetable', 'dipping sauce', 'cracker crumbs', 'cream cheese', 'sour cream', 'lemon juice', 'caramel ice cream', 'potato', 'ginger', 'garam masala', 'frozen peas', 'cilantro', 'wonton wrappers', 'yogurt', 'lime zest', 'lime juice', 'zucchini', 'summer squash', 'cremini mushrooms', 'cauliflow

In [ ]:
exploded_df = key_df['NA_removed_keys'].explode()

# Remove square brackets and split each string in the lists, then explode
words = exploded_df.str.replace("[\[\]']", "").str.split(', ').explode()

# Remove any extra white spaces and make lowercase for consistent counting
words = words.str.strip().str.lower()

# Calculate word frequency
word_freq = words.value_counts()

# Convert the Series to a DataFrame
word_freq_df = word_freq.reset_index()
word_freq_df.columns = ['Word', 'Count']

print(word_freq_df)

<ipython-input-72-83560878cfe6>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  words = exploded_df.str.replace("[\[\]']", "").str.split(', ').explode()


                                Word  Count
0                               salt   5292
1                              sugar   5048
2                              onion   4773
3                                oil   4428
4                             butter   4250
...                              ...    ...
1743                       marmalade      1
1744  arge rutabaga peeled and diced      1
1745              citric acid powder      1
1746        frozen pound cake thawed      1
1747               mini bagels split      1

[1748 rows x 2 columns]


In [ ]:
test_data = data['ingredients'].tolist()

## 3.3.Key word list construction

In [ ]:
key_df

,NA_removed_values,NA_removed_keys
0,"['1/4 cup', '1/4 teaspoon', '1/8 teaspoon', '1...","['flour', 'salt', 'pepper', 'egg', 'bread', 'p..."
1,"['4 cups', '1/4 cup', '2 tablespoons', '2 teas...","['french fries', 'oil', 'ranch', 'bacon', 'sal..."
2,"['4', '1/4 cup', '3 tablespoons', '1/2 teaspoo...","['oil', 'flour', 'cocoa powder', 'baking powde..."
3,"['2 (10-ounces) pkg.', '1/2 cup', '1/2 cup', '...","['spinach', 'feta cheese', 'mozzarella', 'parm..."
4,"['2/3 cup', '1 teaspoon', '1/2 teaspoon', '2 t...","['pecan', 'rosemary', 'salt', 'flour', 'egg', ..."
...,...,...
13592,"['1 (16 ounce) can', '1 pound', '1 (1.25 ounce...","['bean', 'beef', 'season', 'oil', 'tortilla', ..."
13593,"['8 medium', '1 (11 ounce) can', '½ cup', '½ c...","['cherry', 'pizza crust', 'mozzarella', 'pizza..."
13594,"['8', '¼ cup', '⅓ cup', '16']","['mini bagels split', 'pizza sauce', 'cheese b..."
13595,"['¼ cup', '1 cup', '1', '1 pinch', '2 cups', '...","['flour', 'water', 'yeast', 'sugar', 'pizza sa..."


In [ ]:
def formatting_for_converted_unit(path):
  df = pd.read_csv(path)
  # Remove the word "grams" from all entries
  df['converted_ingredient_info'] = df['converted_ingredient_info'].str.replace(' grams', '', regex=False)

  # Create an enumeration column for each ingredient per dish_index
  df['ingredient_num'] = df.groupby('dish_index').cumcount() + 1

  # Pivot the table
  pivoted = df.pivot_table(index=['dish_index'], columns='ingredient_num', values='converted_ingredient_info', aggfunc='first').reset_index()

  # Rename the columns for clarity
  ingredient_cols = ['ingredient_' + str(col) for col in pivoted.columns[1:]]
  pivoted.columns = ['dish_index'] + ingredient_cols

  # Combine the ingredient columns into a single column
  pivoted['combined_ingredients'] = pivoted[ingredient_cols].apply(lambda row: list(row.dropna()), axis=1)

  # Split numbers and ingredients into two separate lists
  pivoted['quantities'] = pivoted['combined_ingredients'].apply(lambda x: [item.split(' ')[0] for item in x])
  pivoted['ingredients'] = pivoted['combined_ingredients'].apply(lambda x: [' '.join(item.split(' ')[1:]) for item in x])

  # Drop the individual ingredient columns and the combined_ingredients column
  pivoted = pivoted[['dish_index', 'quantities','ingredients']]
  return pivoted

df = formatting_for_converted_unit(path)

In [ ]:
df

,dish_index,quantities,ingredients
0,1,"[1.0, 31.25, 1.5, 0.24, 50.0, 59.15, 29.57, 10...","[oil, flour, salt, pepper, egg, bread, bread, ..."
1,2,"[28.25, 0.1, 30.0, 9.86, 946.35, 56.0]","[cheddar cheese, salt, ranch, bacon, french fr..."
2,3,"[2.0, 1.0, 31.25, 15.0, 1.5, 1.0, 44.36, 42.0,...","[baking powder, oil, flour, cocoa powder, salt..."
3,4,"[1.0, 1.0, 113.5, 453.59, 3.0, 56.0, 75.0, 566...","[dill, tzatziki sauce, butter, dough, garlic, ..."
4,5,"[453.59, 1.0, 14.0, 50.0, 66.33, 3.0, 0.1, 15.0]","[trout, emon, maple syrup, egg, pecan, salt, r..."
...,...,...,...
13574,13593,"[8.0, 100.5, 220.0, 246.0, 120.0, 113.4, 720.0...","[olives, avocado, onion, tomato, cream, green ..."
13575,13594,"[3.0, 1.0, 1.0, 8.0, 56.0, 39.43, 122.5, 311.8...","[garlic, pepper, season, pepper, mozzarella, m..."
13576,13595,"[41214.08, 336.0, 61.25, 36.96]","[turkey, mini bagels split, pizza sauce, chees..."
13577,13596,"[1.0, 1.0, 8.0, 490.0, 20.0, 1.0, 1.0, 236.59,...","[salt, oil, basil, pizza sauce, mozzarella, su..."


# 4.Unit conversion: US unit measurement to Mg/grams


## 4.1.split into 130 datasets （1000 rows per dataset）

In [ ]:
import requests
import pandas as pd
from ast import literal_eval
from fractions import Fraction
import unicodedata

In [ ]:
import requests
import pandas as pd
import time

def convert_to_grams(row, max_retries=3):
    API_KEY = "ac1a9733e542439c95cb5d4b08de0d1c"
    endpoint = "https://api.spoonacular.com/recipes/convert"
    params = {
        "ingredientName": row['ingredient'],
        "sourceAmount": row['quantity'],
        "sourceUnit": row['unit'] if row['unit'] != 'NA' else '',
        "targetUnit": "grams",
        "apiKey": API_KEY
    }

    # Initialize retry count
    retries = 0

    while retries < max_retries:
        try:
            response = requests.get(endpoint, params=params)
            response.raise_for_status()

            # Parse the JSON response
            data = response.json()

            # Get the target amount in grams
            target_amount = data['targetAmount']

            # Create a dictionary to store the converted values along with the dish index
            result = {
                'dish_index': row['dish_index'],
                'converted_ingredient_info': f"{target_amount} grams {row['ingredient']}"
            }
            # Ensure a delay of just over 1 second to not exceed the rate limit
            time.sleep(1.05)
            return result

        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:  # Too Many Requests
                wait_time = (2 ** retries)  # Exponential backoff
                print(f"Rate limit reached. Waiting for {wait_time} seconds...")
                time.sleep(wait_time)
                retries += 1
            else:
                print(f"Failed due to: {e}")
                break

        except requests.exceptions.RequestException as e:
            print(f"Failed to convert ingredient: {e}")
            break

    # Return original ingredient info if all retries failed
    result = {
        'dish_index': row['dish_index'],
        'converted_ingredient_info': f"{row['quantity']} {row['unit']} {row['ingredient']}"
    }
    return result


## 4.2.Ingredient Value Storage

### ingredient_1 coversion

In [ ]:
ingredient_1 = pd.read_excel('split/split_1.xlsx', sheet_name='Sheet1')

# Apply the function to each row in the DataFrame
ingredient_1 = ingredient_1.apply(convert_to_grams, axis=1)

# Create a new DataFrame from the converted data
ingredient_1 = pd.DataFrame(list(ingredient_1))

# Print the new DataFrame to check the results
print(ingredient_1)

In [ ]:
# Saving the new DataFrame to a new Excel file
output_filepath = 'ingredient_1 coversion.csv'
ingredient_1.to_csv(output_filepath, index=False)

### ingredient_2coversion

In [ ]:
ingredient_2 = pd.read_excel('split/split_2.xlsx', sheet_name='Sheet1')

# Apply the function to each row in the DataFrame
ingredient_2 = ingredient_2.apply(convert_to_grams, axis=1)

# Create a new DataFrame from the converted data
ingredient_2 = pd.DataFrame(list(ingredient_2))

# Print the new DataFrame to check the results
print(ingredient_2)

     dish_index  converted_ingredient_info
0           116          14.0 grams butter
1           116           55.0 grams onion
2           116           9.0 grams garlic
3           116     120.0 grams white wine
4           116         414.0 grams cherry
..          ...                        ...
994         225         1.0 grams cinnamon
995         225    0.5 grams ground nutmeg
996         225  0.5 grams ground allspice
997         225     191.25 grams pie crust
998         225          28.0 grams butter

[999 rows x 2 columns]


In [ ]:
# Saving the new DataFrame to a new Excel file
output_filepath = 'ingredient_2 coversion.csv'
ingredient_2.to_csv(output_filepath, index=False)

### ingredient_3 coversion

In [ ]:
ingredient_3 = pd.read_excel('split/split_3.xlsx', sheet_name='Sheet1')

# Apply the function to each row in the DataFrame
ingredient_3 = ingredient_3.apply(convert_to_grams, axis=1)

# Create a new DataFrame from the converted data
ingredient_3 = pd.DataFrame(list(ingredient_3))

# Print the new DataFrame to check the results
print(ingredient_3)

     dish_index converted_ingredient_info
0           225             1.0 grams oil
1           225             1.0 grams oat
2           225         100.0 grams sugar
3           225           4.93 grams emon
4           225        1.0 grams cinnamon
..          ...                       ...
994         350  2.46 grams pico de gallo
995         351         119.0 grams cream
996         351         119.0 grams cream
997         351       283.5 grams spinach
998         351    184.27 grams artichoke

[999 rows x 2 columns]


In [ ]:
# Saving the new DataFrame to a new Excel file
output_filepath = 'ingredient_3 coversion.csv'
ingredient_3.to_csv(output_filepath, index=False)

### ingredient_4 coversion

In [ ]:
ingredient_4 = pd.read_excel('split/split_4.xlsx', sheet_name='Sheet1')

# Apply the function to each row in the DataFrame
ingredient_4 = ingredient_4.apply(convert_to_grams, axis=1)

# Create a new DataFrame from the converted data
ingredient_4 = pd.DataFrame(list(ingredient_4))

# Print the new DataFrame to check the results
print(ingredient_4)

     dish_index   converted_ingredient_info
0           351      112.0 grams mozzarella
1           351  50.0 grams parmesan cheese
2           351            3.0 grams garlic
3           351              1.5 grams salt
4           351           226.8 grams bread
..          ...                         ...
994         458       2.0 grams baking soda
995         458              3.0 grams salt
996         459          113.5 grams butter
997         459           100.0 grams sugar
998         459          354.0 grams banana

[999 rows x 2 columns]


In [ ]:
# Saving the new DataFrame to a new Excel file
output_filepath = 'ingredient_4 coversion.csv'
ingredient_4.to_csv(output_filepath, index=False)

### ingredient_5 coversion

In [ ]:
ingredient_5 = pd.read_excel('split/split_5.xlsx', sheet_name='Sheet1')

# Apply the function to each row in the DataFrame
ingredient_5 = ingredient_5.apply(convert_to_grams, axis=1)

# Create a new DataFrame from the converted data
ingredient_5 = pd.DataFrame(list(ingredient_5))

# Print the new DataFrame to check the results
print(ingredient_5)

In [ ]:
# Saving the new DataFrame to a new Excel file
output_filepath = 'ingredient_5 coversion.csv'
ingredient_5.to_csv(output_filepath, index=False)

### ingredient_6 coversion

In [ ]:
ingredient_6 = pd.read_excel('split/split_6.xlsx', sheet_name='Sheet1')

# Apply the function to each row in the DataFrame
ingredient_6 = ingredient_6.apply(convert_to_grams, axis=1)

# Create a new DataFrame from the converted data
ingredient_6 = pd.DataFrame(list(ingredient_6))

# Print the new DataFrame to check the results
print(ingredient_6)

In [ ]:
# Saving the new DataFrame to a new Excel file
output_filepath = 'ingredient_6 coversion.csv'
ingredient_6.to_csv(output_filepath, index=False)

# 5/6. Ingredient name consisitency and Dimension reduction

convertion from string to list, with text pre-processing

In [ ]:
# copy paste the top 150 values from frequency excel sheet
pd_word_list['word_list']  = pd.read_csv(io.StringIO('''
salt
sugar
onion
oil
butter
garlic
pepper
flour
cheese
cream
milk
sauce
chicken
egg
tomato
cinnamon
lemon
beef
baking powder
vinegar
potato
berry
parsley
carrot
baking soda
celery
bread
bean
apple
season
ginger
rice
mushroom
lime
cumin
paprika
oregano
chocolate
basil
mustard
mayonnaise
ground nutmeg
cilantro
almond
thyme
orange
sage
bacon
clove
oat
pork
honey
cornstarch
rum
mozzarella
walnut
bay
chili powder
beaten
coconut
jalapeno
spinach
pecan
pasta
tortilla
peanut
yolk
yeast
shrimp
raisin
ham
vegetable
noodle
avocado
rosemary
cocoa powder
margarine
ketchup
shorten
white wine
cabbage
chile
tomato paste
cherry
olive
zucchini
dill
banana
lightly beaten
cucumber
white
turkey
pie crust
broccoli
steak
salsa
ettuce
eek
sesame seed
halfandhalf
shallot
spaghetti
maple syrup
meat
coriander
bouillon
tuna
mint
corn syrup
molass
ground allspice
red wine
yogurt
pumpkin puree
turmeric
yellow mustard
pear
whipped topping
salad dressing
curry powder
ale
crab
chive
cream of tartar
marshmallow
peach
macaroni
salmon
crust pie
bun
sriracha
cake mix
pumpkin pie spice
beer
lentil
tofu
rhubarb
cornmeal
radish
artichoke
wrapper
ranch
pastry
cardamom
dough
cauliflower
frozen pea
tortilla chip
kernel corn
'''), header=None)


In [ ]:
def mapping_tool(word_list):
  # for format conversion from string data to a list
  processed_text_list = []
  for i in range(len(word_list)):
    In_list_form = ast.literal_eval(word_list[i])
    #print(f'sentence: {In_list_form}')
    lower_list =[]
    # text pre-processing: lower case, punctuation removal
    for w in In_list_form:
      w = w.lower().strip()
      lower_list.append(w)
    symbols_to_remove = [',', '.', '!', '-']
    cleaned_list = []
    for item in lower_list:
        for symbol in symbols_to_remove:
          #print(f'before remove{item}')
          item = item.replace(symbol, '')
          #print(f'after remove{item}')
        cleaned_list.append(item)
    processed_text_list.append(cleaned_list)

  finalised_list = []
  for item in processed_text_list:
    result = []
    for i in item:
      replacement = i
      for keyword in Words:
          if keyword in i:
              replacement = keyword
          else:
            continue
      result.append(replacement)
    finalised_list.append(result)
  return finalised_list

In [ ]:
text_list = mapping_tool(test_data)
print(text_list)

[['flour', 'salt', 'pepper', 'egg', 'bread', 'bread', 'parmesan cheese', 'hearts of palm', 'olive'], ['french fries', 'olive', 'ranch', 'bacon', 'salt', 'cheddar cheese'], ['olive', 'flour', 'cocoa powder', 'baking powder', 'baking soda', 'salt', 'butter', 'apple', 'oil', 'coffee powder', 'nil', 'egg', 'sugar', 'butter', 'honey', 'nil', 'sugar', 'water'], ['spinach', 'feta cheese', 'mozzarella', 'parmesan cheese', 'garlic', 'dough', 'butter', 'tzatziki sauce', 'dill'], ['pecan', 'rosemary', 'salt', 'flour', 'egg', 'trout', 'maple syrup', 'emon'], ['milk', 'egg', 'flour', 'salt', 'pepper', 'paprika', 'pepper', 'oregano', 'thyme', 'cumin', 'onion', 'mayonnaise', 'radish', 'ketchup', 'paprika', 'salt', 'oregano', 'pepper', 'pepper'], ['white', 'water', 'flour', 'season', 'salt', 'pepper', 'vegetable', 'dipping sauce'], ['rum', 'sugar', 'butter', 'cream', 'cream', 'sugar', 'emon', 'nil', 'salt', 'egg', 'cream', 'cream', 'pecan'], ['potato', 'olive', 'onion', 'ginger', 'garlic', 'garam masa

In [ ]:
#Convert to df
df = pd.DataFrame([', '.join(f'"{item}"' for item in sublist) for sublist in text_list], columns=['Combined'])


In [ ]:
print(df)

                                                Combined
0      "flour", "salt", "pepper", "egg", "bread", "br...
1      "french fries", "olive", "ranch", "bacon", "sa...
2      "olive", "flour", "cocoa powder", "baking powd...
3      "spinach", "feta cheese", "mozzarella", "parme...
4      "pecan", "rosemary", "salt", "flour", "egg", "...
...                                                  ...
13592  "bean", "beef", "season", "oil", "tortilla", "...
13593  "cooking spray", "cherry", "pizza crust", "moz...
13594  "mini bagels split", "pizza sauce", "cheese bl...
13595  "flour", "salt", "water", "yeast", "white", "f...
13596  "bacon", "onion", "garlic", "cumin", "oregano"...

[13597 rows x 1 columns]


In [ ]:
# Check for items with more than two words
more_than_two = []
for i in range(len(final_list)):
  result = [item for item in final_list[i] if len(item.split()) > 3]
  if result == []:
    continue
  else:
    more_than_two.append(result)
print(more_than_two)
print(len(more_than_two))

[['fresh or frozen raspberries'], ['fresh black raspberries or blackberries'], ['1/2 cups frozen black raspberries'], ['imes thinly sliced divided'], ['cherries pitted and halved'], ['frozen raspberries in syrup thawed and undrained'], ['¾ cup fresh blueberries', '¾ cup fresh blackberries', '¾ cup fresh raspberries'], ['½ tablespoons dried cranberries'], ['ime cut into wedges (optional)'], ['hulled and halved strawberries'], ['vegan shredded parmesanstyle cheese'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork'], ['baked beans with pork', 'sweet and sour sauce'], ['baked beans with pork'], ['to 110 degrees f (40 to 45 degrees c)'], [' beaten at room temperature'], ['¾ cup fatfree halfandhalf'], ['dried split peas rinsed'], ['imes cut into wedges'], ['fivespice powder or to taste (optional)'], ['

In [ ]:
df

,dish_index,quantities,ingredients,filtered_values,filtered_keys,vector
0,1,"[1.0, 31.25, 1.5, 0.24, 50.0, 59.15, 29.57, 10...","[oil, flour, salt, pepper, egg, bread, bread, ...","[1.0, 31.25, 1.5, 0.24, 50.0, 59.15, 29.57, 10.0]","[oil, flour, salt, pepper, egg, bread, bread, ...","[1.5, 0, 0, 1.0, 0, 0, 0.24, 31.25, 10.0, 0, 0..."
1,2,"[28.25, 0.1, 30.0, 9.86, 946.35, 56.0]","[cheese, salt, ranch, bacon, french fry, oil]","[28.25, 0.1, 9.86, 56.0]","[cheese, salt, bacon, oil]","[0.1, 0, 0, 56.0, 0, 0, 0, 0, 28.25, 0, 0, 0, ..."
2,3,"[2.0, 1.0, 31.25, 15.0, 1.5, 1.0, 44.36, 42.0,...","[baking powder, oil, flour, cocoa powder, salt...","[2.0, 1.0, 31.25, 1.5, 1.0, 44.36, 42.0, 150.0...","[baking powder, oil, flour, salt, baking soda,...","[1.5, 150.0, 0, 1.0, 42.0, 0, 0, 31.25, 0, 0, ..."
3,4,"[1.0, 1.0, 113.5, 453.59, 3.0, 56.0, 75.0, 566...","[dill, sauce, butter, dough, garlic, mozzarell...","[1.0, 113.5, 3.0, 56.0, 75.0, 50.0]","[sauce, butter, garlic, mozzarella, cheese, ch...","[0, 0, 0, 0, 113.5, 3.0, 0, 0, 75.0, 0, 0, 1.0..."
4,5,"[453.59, 1.0, 14.0, 50.0, 66.33, 3.0, 0.1, 15.0]","[trout, lemon, maple syrup, egg, pecan, salt, ...","[1.0, 50.0, 3.0, 15.0]","[lemon, egg, salt, flour]","[3.0, 0, 0, 0, 0, 0, 0, 15.0, 0, 0, 0, 0, 0, 5..."
...,...,...,...,...,...,...
13574,13593,"[8.0, 100.5, 220.0, 246.0, 120.0, 113.4, 720.0...","[olive, avocado, onion, tomato, cream, chile, ...","[220.0, 246.0, 120.0, 14.0, 453.59, 453.59, 22...","[onion, tomato, cream, oil, beef, bean, cheese]","[0, 0, 220.0, 14.0, 0, 0, 0, 0, 226.8, 120.0, ..."
13575,13594,"[3.0, 1.0, 1.0, 8.0, 56.0, 39.43, 122.5, 311.8...","[garlic, pepper, season, pepper, mozzarella, m...","[3.0, 1.0, 8.0, 56.0, 39.43, 122.5, 16.0]","[garlic, pepper, pepper, mozzarella, mozzarell...","[0, 0, 16.0, 0, 0, 3.0, 1.0, 0, 0, 0, 0, 122.5..."
13576,13595,"[41214.08, 336.0, 61.25, 36.96]","[turkey, mini bagels split, sauce, cheese]","[61.25, 36.96]","[sauce, cheese]","[0, 0, 0, 0, 0, 0, 0, 0, 36.96, 0, 0, 61.25, 0..."
13577,13596,"[1.0, 1.0, 8.0, 490.0, 20.0, 1.0, 1.0, 236.59,...","[salt, oil, basil, sauce, mozzarella, sugar, y...","[1.0, 1.0, 490.0, 20.0, 1.0, 6.0, 3.0, 31.25]","[salt, oil, sauce, mozzarella, sugar, salt, fl...","[1.0, 1.0, 0, 1.0, 0, 0, 0, 3.0, 0, 0, 0, 490...."


In [ ]:
# Finding the missing numbers
def find_missing_numbers(numbers):
    full_sequence = set(range(min(numbers), max(numbers) + 1))
    return sorted(list(full_sequence - set(numbers)))

missing_numbers = find_missing_numbers(df['dish_index'])
print(missing_numbers)

[7877, 8311, 8320, 9914, 9933, 11223, 11260, 11262, 11264, 11277, 11290, 11790, 12177, 12465, 12995, 13002, 13006, 13424]


In [ ]:
import inflect

def correct_word(word):
    p = inflect.engine()

    # If word is empty or not a string, return as is
    if not word or not isinstance(word, str):
        return word

    # Correct for 'ime' and 'emon' errors
    if 'ime' in word:
        word = word.replace('ime', 'lime')
    if 'emon' in word:
        word = word.replace('emon', 'lemon')

    # Convert plural to singular
    singular_word = p.singular_noun(word)
    if singular_word:
        return singular_word
    return word

def correct_list(ingredient_str):
    # Convert the string representation of list to an actual list

    # Correct each word/phrase in the list
    corrected_list = [correct_word(word) for word in ingredient_str]

    return corrected_list

df['ingredients'] = df['ingredients'].apply(correct_list)


In [ ]:
# Keyword list to match and replace
key_list = ['pork', 'beef', 'berry', 'chilli', 'peas', 'chicken', 'lime', 'lemon', 'cheese', 'pepper', 'vinegar','season','chile','oil','bean','popcorn','rib','sauce', 'grape', 'olive','flour', ]

def replace_phrase(ingredients):
    new_keys = []
    for ingredient in ingredients:
        replaced = False
        for keyword in key_list:
            if keyword in ingredient:
                new_keys.append(keyword)
                replaced = True
                break
        if not replaced:
            new_keys.append(ingredient)
    return new_keys

# Applying the function on the dataframe
df['ingredients'] = df['ingredients'].apply(replace_phrase)


In [ ]:
df

,dish_index,quantities,ingredients
0,1,"[1.0, 31.25, 1.5, 0.24, 50.0, 59.15, 29.57, 10...","[oil, flour, salt, pepper, egg, bread, bread, ..."
1,2,"[28.25, 0.1, 30.0, 9.86, 946.35, 56.0]","[cheese, salt, ranch, bacon, french fry, oil]"
2,3,"[2.0, 1.0, 31.25, 15.0, 1.5, 1.0, 44.36, 42.0,...","[baking powder, oil, flour, cocoa powder, salt..."
3,4,"[1.0, 1.0, 113.5, 453.59, 3.0, 56.0, 75.0, 566...","[dill, sauce, butter, dough, garlic, mozzarell..."
4,5,"[453.59, 1.0, 14.0, 50.0, 66.33, 3.0, 0.1, 15.0]","[trout, lemon, maple syrup, egg, pecan, salt, ..."
...,...,...,...
13574,13593,"[8.0, 100.5, 220.0, 246.0, 120.0, 113.4, 720.0...","[olive, avocado, onion, tomato, cream, chile, ..."
13575,13594,"[3.0, 1.0, 1.0, 8.0, 56.0, 39.43, 122.5, 311.8...","[garlic, pepper, season, pepper, mozzarella, m..."
13576,13595,"[41214.08, 336.0, 61.25, 36.96]","[turkey, mini bagels split, sauce, cheese]"
13577,13596,"[1.0, 1.0, 8.0, 490.0, 20.0, 1.0, 1.0, 236.59,...","[salt, oil, basil, sauce, mozzarella, sugar, y..."


In [ ]:
# prompt: how to convert pd column to a single list

word_list_100 = pd_word_10['customised'].to_list()


In [ ]:
# Function to filter key-value pairs based on key_list
def filter_pairs(values, keys):
    new_values = [value for value, key in zip(values, keys) if key in word_list_100]
    new_keys = [key for key in keys if key in word_list_100]
    return new_values, new_keys

# Apply the function
df['filtered_values'], df['filtered_keys'] = zip(*df.apply(lambda row: filter_pairs(row['quantities'], row['ingredients']), axis=1))

print(df[['filtered_values', 'filtered_keys']])

                                         filtered_values  \
0      [1.0, 31.25, 1.5, 0.24, 50.0, 59.15, 29.57, 10.0]   
1                               [28.25, 0.1, 9.86, 56.0]   
2      [2.0, 1.0, 31.25, 1.5, 1.0, 44.36, 42.0, 150.0...   
3                    [1.0, 113.5, 3.0, 56.0, 75.0, 50.0]   
4                                 [1.0, 50.0, 3.0, 15.0]   
...                                                  ...   
13574  [220.0, 246.0, 120.0, 14.0, 453.59, 453.59, 22...   
13575          [3.0, 1.0, 8.0, 56.0, 39.43, 122.5, 16.0]   
13576                                     [61.25, 36.96]   
13577      [1.0, 1.0, 490.0, 20.0, 1.0, 6.0, 3.0, 31.25]   
13578  [56.5, 112.0, 59.5, 176.0, 1.0, 6.0, 110.11, 2...   

                                           filtered_keys  
0      [oil, flour, salt, pepper, egg, bread, bread, ...  
1                             [cheese, salt, bacon, oil]  
2      [baking powder, oil, flour, salt, baking soda,...  
3      [sauce, butter, garlic, mozzarella, 

In [ ]:
# Additional processing (as in your original function)
def get_vector(row):
    ingredients = row['filtered_keys']
    quantities = row['filtered_values']
    vector = []

    for key in word_list_100:
        if key in ingredients:
            index = ingredients.index(key)
            vector.append(quantities[index])
        else:
            vector.append(0)

    return vector

In [ ]:
# Apply the function to each row in the DataFrame
df['vector'] = df.apply(get_vector, axis=1)

In [ ]:
len(df['vector'][0])

36

# 7.Final formatting

## 7.1.Image formatting

In [ ]:
import json
import ast
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path = '/content/drive/My Drive/UOS/DATA5703/data/dishimg'
# Get all JSON files in the directory
all_json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]


batch_files = all_json_files[11000:12000] #改这里

data_list = []

# Loop through each JSON file in the batch
for json_file in batch_files:
    with open(os.path.join(folder_path, json_file), 'r') as file:
        # Load the content of the JSON file
        data = json.load(file)

        # Check if the loaded data is a list or a dictionary
        if isinstance(data, list):
            # If it's a list, convert it to a dictionary with integer keys
            data = {i: item for i, item in enumerate(data)}

        # Append the filename (without .json) and the data to the data_list
        data_list.append((json_file.replace('.json', ''), data))


In [ ]:
# Convert the list to a DataFrame
df = pd.DataFrame(data_list, columns=['Image_Name', 'Content'])

In [ ]:
df['Content'] = df['Content'].apply(lambda x: [item for sublist in x.values() for item in sublist])

In [ ]:
row_value = df['Content'].iloc[4]

# Dimension
outer_length = len(row_value)
inner_length = len(row_value[0]) if outer_length > 0 else 0

print(f"The dimension is: {outer_length} x {inner_length}")

The dimension is: 60025 x 3


In [ ]:

excel_file_path = '/content/drive/My Drive/UOS/DATA5703/data/mapping_1.xlsx'

# Save the DataFrame to an Excel file
filtered_df.to_excel(excel_file_path, index=False)

print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /content/drive/My Drive/UOS/DATA5703/data/mapping_1.xlsx


In [ ]:
from google.colab import files

# Save your DataFrame or any other data
df.to_csv('data_11000_12000.csv', index=False)  # As an example, if you're working with a pandas DataFrame

# Download the file to your local system
files.download('data_11000_12000.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 7.2.final formatting: combine image data with values

In [ ]:
path = "/content/Data_final.xlsx"
df_value = pd.read_excel(path, engine='openpyxl')

In [ ]:
df_value

,name,value_150,Value_100,value_50,Value_30,value_cus_36
0,Air Fryer Hearts of Palm Sticks,"['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1..."
1,Air Fryer Cheesy Bacon Ranch French Fries,"['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ..."
2,Air Fryer Mini Dark Chocolate Cake with Brown ...,"['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2..."
3,Air Fryer Spanakopita,"[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ..."
4,Air Fryer Pecan Crusted Trout,"['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
13573,True Cheeseburger Pizza,"['1.5', 0, '80.0', 0, 0, 0, '0.5', 0, '113.0',...","['1.5', 0, '80.0', 0, 0, 0, '0.5', 0, '113.0',...","['1.5', 0, '80.0', 0, 0, 0, '0.5', 0, '113.0',...","['1.5', 0, '80.0', 0, 0, 0, '0.5', 0, '113.0',...","['1.5', 0, '80.0', 0, 0, 0, '0.5', 0, '113.0',..."
13574,Mexican Pizza I,"[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '..."
13575,Pizza Cupcakes,"[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,..."
13576,Easy Mini Bagel Pizzas,"[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, '61.25..."


In [ ]:
df_value.rename(columns={'dish name': 'name'}, inplace=True)

In [ ]:
path_1 = "/content/Image_data.xlsx"
df_image = pd.read_excel(path_1, engine='openpyxl')

In [ ]:
df_image

,name,Content
0,Sloppy Joes for a Crowd,"[[[177, 155, 144], [183, 161, 150], [187, 165,..."
1,Tonya's Terrific Sloppy Joes,"[[[244, 170, 103], [239, 164, 98], [242, 164, ..."
2,Sloppy Joe Mamas,"[[[121, 63, 59], [111, 58, 48], [112, 66, 49],..."
3,Ruby Drive Sloppy Joes,"[[[148, 131, 112], [139, 122, 103], [134, 115,..."
4,Nonie's Best BBQ,"[[[28, 26, 29], [28, 26, 29], [29, 27, 30], [2..."
...,...,...
12310,Artichoke Dip with Fresh Spinach,"[[[139, 117, 90], [182, 134, 83], [182, 148, 1..."
12311,A and Z Dip,"[[[179, 162, 144], [180, 163, 145], [180, 163,..."
12312,"Absolutely Wonderful Cheesy, Creamy Spinach Ar...","[[[120, 76, 38], [119, 72, 35], [124, 70, 35],..."
12313,Healthier Hot Artichoke and Spinach Dip II,"[[[95, 85, 112], [94, 84, 111], [96, 86, 113],..."


In [ ]:
df_image['Content'][15]

'[[[105, 114, 124], [106, 114, 125], [104, 110, 122], [105, 110, 122], [102, 111, 121], [102, 112, 120], [106, 113, 121], [107, 112, 119], [104, 113, 121], [102, 112, 121], [102, 112, 121], [103, 113, 122], [103, 114, 120], [102, 113, 121], [101, 111, 125], [100, 109, 128], [99, 108, 125], [100, 109, 123], [102, 112, 124], [103, 113, 123], [102, 110, 120], [101, 108, 118], [100, 107, 117], [101, 108, 118], [101, 110, 119], [99, 109, 118], [100, 110, 119], [100, 110, 119], [98, 108, 117], [100, 110, 119], [100, 110, 120], [100, 110, 122], [99, 108, 120], [98, 104, 118], [99, 105, 119], [101, 107, 122], [101, 107, 120], [100, 106, 118], [101, 107, 119], [100, 107, 119], [101, 108, 119], [100, 110, 119], [98, 108, 117], [100, 110, 119], [100, 110, 119], [100, 110, 120], [97, 107, 117], [97, 107, 117], [96, 106, 116], [98, 108, 118], [99, 109, 119], [100, 110, 120], [98, 108, 118], [98, 108, 120], [96, 107, 119], [94, 107, 119], [94, 108, 119], [95, 109, 121], [92, 107, 118], [93, 108, 116

In [ ]:
df_image.rename(columns={'Image_Name': 'name'}, inplace=True)

In [ ]:
merged_df = pd.merge(df_value, df_image, on='name', how='inner')

In [ ]:
merged_df

,name,value_150,Value_100,value_50,Value_30,value_cus_36,Content
0,Air Fryer Hearts of Palm Sticks,"['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","['1.5', 0, 0, '1.0', 0, 0, '0.24', '31.25', '1...","[[129, 157, 196], [133, 162, 201], [137, 161, ..."
1,Air Fryer Cheesy Bacon Ranch French Fries,"['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","['0.1', 0, 0, '56.0', 0, 0, 0, 0, '28.25', 0, ...","[[229, 137, 25], [232, 138, 27], [235, 147, 34..."
2,Air Fryer Mini Dark Chocolate Cake with Brown ...,"['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","['1.5', '150.0', 0, '1.0', '42.0', 0, 0, '31.2...","[[128, 101, 90], [138, 111, 100], [154, 127, 1..."
3,Air Fryer Spanakopita,"[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[0, 0, 0, 0, '113.5', '3.0', 0, 0, '75.0', 0, ...","[[73, 56, 36], [66, 50, 29], [74, 57, 42], [64..."
4,Air Fryer Pecan Crusted Trout,"['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","['3.0', 0, 0, 0, 0, 0, 0, '15.0', 0, 0, 0, 0, ...","[[244, 244, 244], [246, 246, 246], [247, 247, ..."
...,...,...,...,...,...,...,...
12222,BBQ Pulled Pork Pizza,"[0, 0, '110.11', 0, 0, 0, '1.0', 0, 0, 0, 0, 0...","[0, 0, '110.11', 0, 0, 0, '1.0', 0, 0, 0, 0, 0...","[0, 0, '110.11', 0, 0, 0, '1.0', 0, 0, 0, 0, 0...","[0, 0, '110.11', 0, 0, 0, '1.0', 0, 0, 0, 0, 0...","[0, 0, '110.11', 0, 0, 0, '1.0', 0, 0, 0, 0, '...","[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [..."
12223,Mexican Pizza I,"[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[0, 0, '220.0', '14.0', 0, 0, 0, 0, '226.8', '...","[[36, 37, 41], [44, 46, 50], [56, 59, 63], [62..."
12224,Pizza Cupcakes,"[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[0, 0, '16.0', 0, 0, '3.0', '1.0', 0, 0, 0, 0,...","[[227, 76, 81], [222, 71, 76], [233, 82, 85], ..."
12225,Easy Mini Bagel Pizzas,"[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, 0, '61...","[0, 0, 0, 0, 0, 0, 0, 0, '36.96', 0, 0, '61.25...","[[114, 136, 125], [90, 111, 106], [73, 94, 92]..."


In [ ]:
merged_df['Content'] = merged_df['Content'].apply(lambda x: x[1:] + ']]')

In [ ]:
merged_df['Content'][5]

'[[154, 98, 97], [154, 98, 97], [158, 98, 98], [172, 121, 118], [196, 173, 161], [231, 220, 209], [135, 97, 98], [102, 35, 42], [112, 43, 46], [108, 45, 46], [108, 45, 46], [109, 44, 46], [103, 39, 40], [102, 38, 33], [101, 38, 31], [100, 37, 30], [115, 52, 45], [147, 84, 77], [184, 134, 123], [209, 165, 151], [217, 172, 161], [220, 173, 177], [223, 176, 181], [221, 176, 176], [218, 178, 171], [219, 178, 172], [222, 181, 180], [222, 180, 181], [221, 179, 178], [221, 180, 174], [221, 180, 174], [221, 180, 178], [221, 179, 180], [221, 179, 179], [221, 180, 174], [221, 180, 174], [221, 180, 174], [221, 180, 174], [221, 180, 174], [218, 182, 174], [215, 182, 173], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 180, 171], [214, 179, 175], [214, 179, 177], [214, 179, 177], [214, 179, 177], [216, 181, 179], [217, 185, 186], [221, 192, 196], [222, 193, 197], [225, 19

In [ ]:
excel_file_path = '/content/drive/My Drive/UOS/DATA5703/data_Final.xlsx'

# Save the DataFrame to an Excel file
merged_df.to_excel(excel_file_path, index=False)

print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /content/drive/My Drive/UOS/DATA5703/data_Final.xlsx


## 7.3.Data exportation

In [ ]:
#save data to google drive
from google.colab import drive
drive.mount('/content/drive')
excel_file_path = '/content/drive/My Drive/UOS/DATA5703/data_customised.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print(f"DataFrame saved to {excel_file_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DataFrame saved to /content/drive/My Drive/UOS/DATA5703/data_customised.xlsx


# 8.Calculator Dataset formatting


In [ ]:
path = "/content/ingredients.csv"
nutrition_df = pd.read_csv(path)

In [ ]:
nutrition_df

,Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,...,Data.Major Minerals.Phosphorus,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,Butter,"Butter, salted",1001,0,158,0,0.06,215,18.8,0.0,...,24,24,643,0.09,684,0.17,0.003,0.0,2.32,7.0
1,Butter,"Butter, whipped, with salt",1002,1,135,6,0.00,225,18.8,0.0,...,24,41,583,0.05,683,0.07,0.008,0.0,1.37,4.6
2,Butter oil,"Butter oil, anhydrous",1003,0,193,0,0.00,256,22.3,0.0,...,3,5,2,0.01,840,0.01,0.001,0.0,2.80,8.6
3,Cheese,"Cheese, blue",1004,0,74,0,2.34,75,15.4,0.0,...,387,256,1146,2.66,198,1.22,0.166,0.0,0.25,2.4
4,Cheese,"Cheese, brick",1005,0,76,0,2.79,94,15.4,0.0,...,451,136,560,2.60,292,1.26,0.065,0.0,0.26,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,REDUCED SODIUM: Turkey breast,"REDUCED SODIUM: Turkey breast, sliced, prepack...",907081,0,0,0,2.20,49,30.1,0.0,...,249,371,674,0.94,0,0.37,0.410,0.0,0.13,0.0
2328,REDUCED SODIUM: Chicken breast,"REDUCED SODIUM: Chicken breast, deli, rotisser...",907961,0,0,0,2.92,51,44.2,0.0,...,257,360,774,0.51,2,0.14,0.445,0.0,0.32,0.0
2329,REDUCED SODIUM: Bologna,"REDUCED SODIUM: Bologna, meat and poultry",907971,0,0,0,6.31,92,59.3,0.0,...,212,320,1034,1.06,0,0.47,0.193,0.0,0.59,1.7
2330,REDUCED SODIUM: Nuts,"REDUCED SODIUM: Nuts, almond butter, plain, wi...",912695,0,1,0,18.82,0,52.1,10.3,...,508,748,115,3.29,0,0.00,0.103,0.0,24.21,0.0


In [ ]:
# upload the nutrition content sheet from FNDDS
path = "/content/2019-2020 FNDDS At A Glance - FNDDS Ingredients.xlsx"
ingredient_df = pd.read_excel(path)

In [ ]:
selected_columns = ['Ingredient code', 'Ingredient weight (g)']
new_df = ingredient_df[selected_columns]

In [ ]:
new_df

,Ingredient code,Ingredient weight (g)
0,1107,100.0
1,1077,40.0
2,1079,38.0
3,1082,14.0
4,1085,8.0
...,...,...
19303,2046,10.0
19304,4698,25.0
19305,4701,25.0
19306,4699,25.0


In [ ]:
combined_df = new_df.merge(nutrition_df, left_on='Ingredient code', right_on='Nutrient Data Bank Number', how='inner')

In [ ]:
combined_df

,Ingredient code,Ingredient weight (g),Category,Description,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,...,Data.Major Minerals.Phosphorus,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,1107,100.0,Milk,"Milk, human, mature, fluid",1107,0,7,0,6.89,14,...,14,51,17,0.17,61,0.05,0.011,5.0,0.08,0.3
1,1077,40.0,Milk,"Milk, whole, 3.25% milkfat, with added vitamin D",1077,0,7,0,4.67,12,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
2,1077,100.0,Milk,"Milk, whole, 3.25% milkfat, with added vitamin D",1077,0,7,0,4.67,12,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
3,1077,100.0,Milk,"Milk, whole, 3.25% milkfat, with added vitamin D",1077,0,7,0,4.67,12,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
4,1077,244.0,Milk,"Milk, whole, 3.25% milkfat, with added vitamin D",1077,0,7,0,4.67,12,...,101,150,38,0.41,32,0.54,0.061,0.0,0.05,0.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15010,14056,100.0,Beverages,"Beverages, Acai berry drink, fortified",14056,46,5784,23,12.83,0,...,13,130,13,0.11,485,3.40,1.100,42.1,11.11,16.7
15011,4698,25.0,Oil,"Oil, industrial, canola, high oleic",4698,0,0,0,0.00,0,...,0,0,0,0.00,0,0.00,0.000,0.0,17.46,71.3
15012,4701,25.0,Oil,"Oil, industrial, soy, fully hydrogenated",4701,0,0,0,0.00,0,...,0,0,0,0.01,0,0.00,0.000,0.0,8.18,183.9
15013,4699,25.0,Oil,"Oil, industrial, soy, low linolenic",4699,0,0,0,0.00,0,...,0,0,0,0.01,0,0.00,0.000,0.0,8.18,183.9


In [ ]:
path = "/content/mapping.xlsx"
df = pd.read_excel(path)

df = df.drop_duplicates(subset='Category', keep='first')
words_list = pd_word_list['word_list'].tolist()
filtered_df = df[df['Category'].str.contains('|'.join(words_list), case=False, na=False)]